In [1]:
import numpy as np 
import pandas as pd

from sklearn.metrics import hamming_loss,accuracy_score,classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier

import os

from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings("ignore")

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv',header=None)
train_labels_df = pd.read_csv('trainLabels.csv')
submission_df = pd.read_csv('sampleSubmission_small.csv')

In [3]:
print("train_df --> ",train_df.shape)
print("test_df --> ",test_df.shape)
print("train_labels_df --> ",train_labels_df.shape)
print("submission_df --> ",submission_df.shape)

train_df -->  (9999, 146)
test_df -->  (2000, 146)
train_labels_df -->  (49999, 34)
submission_df -->  (990, 2)


In [4]:
train_df.describe

<bound method NDFrame.describe of         id   x1   x2                                            x3  \
0        1   NO   NO  dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=   
1        2  NaN  NaN                                           NaN   
2        3   NO   NO  ib4VpsEsqJHzDiyL0dZLQ+xQzDPrkxE+9T3mx5fv2wI=   
3        4  YES   NO  BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=   
4        5   NO   NO  RTjsrrR8DTlJyaIP9Q3Z8s0zseqlVQTrlSe97GCWfbk=   
...    ...  ...  ...                                           ...   
9994  9995   NO   NO  jComfqYTXYSeH3GvqcOhwPmldb+BCdVJDKKDNkdtw2w=   
9995  9996   NO   NO  Pr5enXjzVzVjZziZxrDcgnyu6CLUftmEbnp6TctyJbU=   
9996  9997  YES  YES  9Avs0tL1zvH7Xx41z2UqrXs11/4IWLmqRAodLt/SKjQ=   
9997  9998   NO   NO  9zkXU3f6YnRPjsWi3lKSCLseIGrleg00tpRI4OplABw=   
9998  9999  YES  YES  MZZbXga8gvaCBqWpzrh2iKdOkcsz/bG/z4BVjUnqWT0=   

                                                x4        x5        x6  \
0     GNjrXXA3SxbgD0dTRblAPO9jFJ7AIaZnu/f48g5XSUk= 

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Columns: 146 entries, id to x145
dtypes: float64(55), int64(31), object(60)
memory usage: 11.1+ MB


In [6]:
train_df.head()

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145
0,1,NO,NO,dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=,GNjrXXA3SxbgD0dTRblAPO9jFJ7AIaZnu/f48g5XSUk=,0.576561,0.073139,0.481394,0.115697,0.472474,...,0.0,0.810,3306,4676,YES,NO,YES,2,0.375535,0.464610
1,2,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.510,4678,3306,YES,NO,YES,4,0.741682,0.593630
2,3,NO,NO,ib4VpsEsqJHzDiyL0dZLQ+xQzDPrkxE+9T3mx5fv2wI=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,1.341803,0.051422,0.935572,0.041440,0.501710,...,0.0,0.850,4678,3306,NO,NO,NO,1,0.776467,0.493159
3,4,YES,NO,BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,0.653912,0.041471,0.940787,0.090851,0.556564,...,0.0,0.945,3306,4678,NO,NO,YES,3,0.168234,0.546582
4,5,NO,NO,RTjsrrR8DTlJyaIP9Q3Z8s0zseqlVQTrlSe97GCWfbk=,3yK2OPj1uYDsoMgsxsjY1FxXkOllD8Xfh20VYGqT+nU=,1.415919,0.000000,1.000000,0.000000,0.375297,...,0.0,1.000,1263,892,NO,NO,NO,1,0.246637,0.361045


In [7]:
test_df.head()

,0,1,2,3,4,5,6,7,8,9,...,136,137,138,139,140,141,142,143,144,145
0,1698001,NO,NO,5KaYd5siHnBD/IjH8BF1fPz5zrCADHZia/Lrhlyxkvc=,FzMc/XY2ETaomhy8gPc9UL8LRkEnQA56+/wVF1fogk8=,1.414798,0.000000,1.000000,0.000000,0.202060,...,0.0,1.000000,1262,892,NO,NO,NO,0,0.089686,0.193344
1,1698002,NO,NO,9ACcuXc7MMm9V7jZSr3P3VxAKyMvLAtsdwPKwgncc+k=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,0.832679,0.049834,0.945938,0.317427,0.482021,...,1.0,0.866667,4672,3311,NO,NO,NO,5,0.945032,0.471318
2,1698003,NO,NO,MeBJ/ZzEIXfNKat4w1oeDxiMNKrAeY0PH41i00hpYDo=,tnLDGLnpYhzsik5+X+WPo4KQJoQA0TfWRlmEtQ3XNJQ=,1.415919,0.000000,1.000000,0.000000,0.703088,...,-1.0,1.000000,1263,892,NO,NO,NO,8,0.557175,0.693587
3,1698004,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.870000,4672,3306,YES,NO,YES,0,0.870538,0.405822
4,1698005,NO,NO,uduY7XWJ8eFgTltv5P0rPh5GW6KwBu+tPFH13uQRN+0=,0L7+hNDV8S57etySgdljbm2AK1zQuLP77lGk2hyEmCo=,1.129212,0.087020,0.814240,1.112804,0.874318,...,0.0,0.870000,4400,3413,YES,NO,YES,2,0.224729,0.870909


In [8]:
train_labels_df.head()

,id,y1,y2,y3,y4,y5,y6,y7,y8,y9,...,y24,y25,y26,y27,y28,y29,y30,y31,y32,y33
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,3,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [9]:
# assigning column name to test dataset
test_df.columns=['id',
                 'x1','x2','x3','x4','x5','x6','x7','x8','x9','x10',
                 'x11','x12','x13','x14','x15','x16','x17','x18','x19','x20',
                 'x21','x22','x23','x24','x25','x26','x27','x28','x29','x30',
                 'x31','x32','x33','x34','x35','x36','x37','x38','x39','x40',
                 'x41','x42','x43','x44','x45','x46','x47','x48','x49','x50',
                 'x51','x52','x53','x54','x55','x56','x57','x58','x59','x60',
                 'x61','x62','x63','x64','x65','x66','x67','x68','x69','x70',
                 'x71','x72','x73','x74','x75','x76','x77','x78','x79','x80',
                 'x81','x82','x83','x84','x85','x86','x87','x88','x89','x90',
                 'x91','x92','x93','x94','x95','x96','x97','x98','x99','x100',
                 'x101','x102','x103','x104','x105','x106','x107','x108','x109','x110',
                 'x111','x112','x113','x114','x115','x116','x117','x118','x119','x120',
                 'x121','x122','x123','x124','x125','x126','x127','x128','x129','x130',
                 'x131','x132','x133','x134','x135','x136','x137','x138','x139','x140',
                 'x141','x142','x143','x144','x145']

In [10]:
test_df.head()

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145
0,1698001,NO,NO,5KaYd5siHnBD/IjH8BF1fPz5zrCADHZia/Lrhlyxkvc=,FzMc/XY2ETaomhy8gPc9UL8LRkEnQA56+/wVF1fogk8=,1.414798,0.000000,1.000000,0.000000,0.202060,...,0.0,1.000000,1262,892,NO,NO,NO,0,0.089686,0.193344
1,1698002,NO,NO,9ACcuXc7MMm9V7jZSr3P3VxAKyMvLAtsdwPKwgncc+k=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,0.832679,0.049834,0.945938,0.317427,0.482021,...,1.0,0.866667,4672,3311,NO,NO,NO,5,0.945032,0.471318
2,1698003,NO,NO,MeBJ/ZzEIXfNKat4w1oeDxiMNKrAeY0PH41i00hpYDo=,tnLDGLnpYhzsik5+X+WPo4KQJoQA0TfWRlmEtQ3XNJQ=,1.415919,0.000000,1.000000,0.000000,0.703088,...,-1.0,1.000000,1263,892,NO,NO,NO,8,0.557175,0.693587
3,1698004,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.870000,4672,3306,YES,NO,YES,0,0.870538,0.405822
4,1698005,NO,NO,uduY7XWJ8eFgTltv5P0rPh5GW6KwBu+tPFH13uQRN+0=,0L7+hNDV8S57etySgdljbm2AK1zQuLP77lGk2hyEmCo=,1.129212,0.087020,0.814240,1.112804,0.874318,...,0.0,0.870000,4400,3413,YES,NO,YES,2,0.224729,0.870909


In [11]:
#checking train dataset rows that has atleast one NAN value
train_df[train_df.isna().any(axis=1)]

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145
1,2,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.51,4678,3306,YES,NO,YES,4,0.741682,0.593630
8,9,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.82,4675,3304,YES,NO,YES,1,0.923426,0.702032
12,13,NO,NO,vIR+5O6D5pKgFw/CFbDtHBZ/AhaNMnxLM4BTIwXUres=,gNVKedeHcKHyWnIMnBCE9t9XOschRQCU8/XfcaY0mMk=,0.440411,0.083787,0.776165,0.253781,0.210774,...,0.0,0.74,4678,3306,YES,NO,YES,0,0.029643,0.199230
25,26,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,1.00,1188,918,YES,NO,YES,5,0.898693,0.502525
26,27,NO,NO,EKynK11V/TnbPzoD8gy12kkm85wTNzN4kciauQQOHO8=,LXzIvxHIblNu5ThzWnaNlPxZrF3BRg4q4JWZPXQ5Z40=,1.415919,0.000000,1.000000,0.000000,0.889153,...,0.0,1.00,1263,892,YES,NO,YES,0,0.032511,0.883610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9978,9979,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,1.00,1262,892,YES,NO,YES,1,0.838565,0.194136
9983,9984,NO,NO,/A8gcPhvt4VEgUwvb8EOaJsx1W/EJcRZvR+LpZQCG5I=,hCXwO/JldK5zcd9ejOD1FwmEgCf96eTdEVy7OtY2Y2g=,1.414798,0.000000,1.000000,0.000000,0.798732,...,0.0,1.00,1262,892,YES,NO,YES,1,0.454036,0.787639
9985,9986,NO,NO,jSCyZ7TLJ2iF5csw4WvfdUDPg0rilDps2Bz6vPTD6K4=,k/m0ZdAeAo2VCPgrn4lo9Aow1dirDyCAmhbX5fvZd7M=,0.793166,0.074085,0.767765,0.617780,0.479581,...,0.0,0.91,4677,3307,NO,NO,NO,0,0.167523,0.470601
9991,9992,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,1.00,1263,893,YES,YES,YES,4,0.731243,0.284244


In [12]:
#checking how many Percentage of data is missing in train dataset where NaN is given
#arranging them in descending order
missing_percent_training_data=train_df.isna().sum().sort_values(ascending=False)*100/len(train_df)
missing_percent_training_data

x26     14.261426
x12     14.261426
x2      14.261426
x3      14.261426
x4      14.261426
          ...    
x53      0.000000
x52      0.000000
x51      0.000000
x50      0.000000
x145     0.000000
Length: 146, dtype: float64

In [13]:
#checking how many Percentage of data is missing in train_label dataset where NaN is given
#arranging them in descending order
missing_percent_train_labels_data=train_labels_df.isna().sum().sort_values(ascending=False)*100/len(train_labels_df)
missing_percent_train_labels_data

id     0.0
y25    0.0
y19    0.0
y20    0.0
y21    0.0
y22    0.0
y23    0.0
y24    0.0
y26    0.0
y1     0.0
y27    0.0
y28    0.0
y29    0.0
y30    0.0
y31    0.0
y32    0.0
y18    0.0
y17    0.0
y16    0.0
y15    0.0
y14    0.0
y13    0.0
y12    0.0
y11    0.0
y10    0.0
y9     0.0
y8     0.0
y7     0.0
y6     0.0
y5     0.0
y4     0.0
y3     0.0
y2     0.0
y33    0.0
dtype: float64

In [14]:
#checking how many Percentage of data is missing in test dataset where NaN is given
#arranging them in descending order
missing_percent_test_data=test_df.isna().sum().sort_values(ascending=False)*100/len(test_df)
missing_percent_test_data

x26     14.7
x12     14.7
x2      14.7
x3      14.7
x4      14.7
        ... 
x53      0.0
x52      0.0
x51      0.0
x50      0.0
x145     0.0
Length: 146, dtype: float64

In [15]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Columns: 146 entries, id to x145
dtypes: float64(55), int64(31), object(60)
memory usage: 11.1+ MB


In [16]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_columns=train_df.select_dtypes(include=numerics).columns.to_list()
len(numeric_columns) 

86

In [17]:
object_columns=train_df.select_dtypes(include=object).columns
len(object_columns)

60

In [18]:
train_df.dtypes.unique()

array([dtype('int64'), dtype('O'), dtype('float64')], dtype=object)

In [19]:
train_df[object_columns]=train_df[object_columns].astype(str)
test_df[object_columns]=test_df[object_columns].astype(str)

In [20]:
train_df[object_columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 60 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   x1      9999 non-null   object
 1   x2      9999 non-null   object
 2   x3      9999 non-null   object
 3   x4      9999 non-null   object
 4   x10     9999 non-null   object
 5   x11     9999 non-null   object
 6   x12     9999 non-null   object
 7   x13     9999 non-null   object
 8   x14     9999 non-null   object
 9   x24     9999 non-null   object
 10  x25     9999 non-null   object
 11  x26     9999 non-null   object
 12  x30     9999 non-null   object
 13  x31     9999 non-null   object
 14  x32     9999 non-null   object
 15  x33     9999 non-null   object
 16  x34     9999 non-null   object
 17  x35     9999 non-null   object
 18  x41     9999 non-null   object
 19  x42     9999 non-null   object
 20  x43     9999 non-null   object
 21  x44     9999 non-null   object
 22  x45     9999 non-null   

In [21]:
boolean_cols = []
alphanumeric_cols = []

for col in object_columns:
    if train_df[col][0].isupper()==True:
        boolean_cols.append(col)
    else:
        alphanumeric_cols.append(col)

In [22]:
alphanumeric_cols

['x3', 'x4', 'x34', 'x35', 'x61', 'x64', 'x65', 'x91', 'x94', 'x95']

In [23]:
train_df['x3'].value_counts()

nan                                             1426
MZZbXga8gvaCBqWpzrh2iKdOkcsz/bG/z4BVjUnqWT0=     290
X/hdUOVR5KuExVGLzjhLcM2CyIqym9t0Nh+ZX05M+1w=     100
3acC0FNTp8BjTkXkLzy9LJSB0FmLpzsJhDtE3riGZ9A=      70
zq3MyXbqpnH+txFzhEWomNfZyeLXubUbZwdaRvKXLs4=      68
                                                ... 
xjmTLdl3cvznW2FwUHzAy1isqqQ/MM75nhLGKaSAEFY=       1
32TnZIoTdTuMO/QEiTndO7R2De+TXxjzsCcGB68PxSs=       1
Zpdzn+mU6g3vVZMqG3IiKWgYk08wwV0eyt7zxX+yew8=       1
l+LC+epkima2ujQczUHf+UXtXJfTXY7r7PDdGSSdSvg=       1
9Avs0tL1zvH7Xx41z2UqrXs11/4IWLmqRAodLt/SKjQ=       1
Name: x3, Length: 4752, dtype: int64

In [24]:
df_object_col = train_df[['x3', 'x4', 'x34', 'x35', 'x61', 'x64', 'x65', 'x91', 'x94', 'x95']]

In [25]:
df_object_col 

,x3,x4,x34,x35,x61,x64,x65,x91,x94,x95
0,dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=,GNjrXXA3SxbgD0dTRblAPO9jFJ7AIaZnu/f48g5XSUk=,mimucPmJSF6NI6KM6cPIaaVxWaQyIQzSgtwTTb9bKlc=,s7mTY62CCkWUFc36AW2TlYAy5CIcniD2Vz+lHzyYCLg=,+2TNtXRI6r9owdGCS80Ia9VVv8ZpuOpVaHEvxRGGu78=,Op+X3asn5H7EQJErI7PR0NkUs3YB+Ld/8OfWuiOC8tU=,GeerC2BbPUcQfQO86NmvOsKrfTvmW7HF+Iru9y+7DPA=,bxU52teuxC05EZyzFihSiKHczE2ZAIVCXekVLG7j3C0=,+dia7tCOijlRGbABX0YKG5L85x/hXLyJwwplN5Qab04=,f4Uu1R9nnf/h03aqiRQT0Fw3WItzNToLCyRlW1Pn8Z8=
1,nan,nan,l0G2rvmLGE6mpPtAibFsoW/0SiNnAuyAc4k35TrHvoQ=,lblNNeOLanWhqgISofUngPYP0Ne1yQv3QeNHqCAoh48=,MZZbXga8gvaCBqWpzrh2iKdOkcsz/bG/z4BVjUnqWT0=,TqL9cs8ZFzALzVpZv6wYBDi+6zwhrdarQE/3FH+XAlA=,aZTF/lredyP4cukeN8bh6kpBjYmS1QFNpPOg2LVm3Lg=,YvZUuCDjLu9VvkCdBWgARWQrvm+FSXgxp0zIrMjcLBc=,dsyhxXKNNJy4WVGD/v4+UGyW3jHWkx2xTdg3STsf34A=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=
2,ib4VpsEsqJHzDiyL0dZLQ+xQzDPrkxE+9T3mx5fv2wI=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,9TRXThP/ifDpJRGFX1LQseibUA1NJ3XM53gy+1eZ46k=,XSJ6E8aAoZC7/KAu3eETpfMg3mCq7HVBFIVIsoMKh9E=,cr+kkNnNFV9YL0vz029hk3ohIDmGuABRVNhFe0ePZyo=,oFsUwSLCWcj8UA1cqILh5afKVcvwlFA+ohJ147Wkz5I=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,mRPnGiKVOWTk/vzZaqlLXZRtdrkcQ/sX0hqBCqOuKq0=,oo9tGpHvTredpg9JkHgYbZAuxcwtSpQxU5mA/zUbxY8=
3,BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,XQG0f+jmjLI0UHAXXH2RYL4MEHa+yd9okO+730PCZuc=,/1yAAEg6Qib4GMD+wvGOlGmpCIPIAzioWtcCwbns9/I=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,Vl+TDNSupucNoI+Fqeo7bMCkxg1hRjgTSS6NYb9BW00=,/1yAAEg6Qib4GMD+wvGOlGmpCIPIAzioWtcCwbns9/I=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=
4,RTjsrrR8DTlJyaIP9Q3Z8s0zseqlVQTrlSe97GCWfbk=,3yK2OPj1uYDsoMgsxsjY1FxXkOllD8Xfh20VYGqT+nU=,XEDyQD4da6aJkZiBf+r7LD2VdhLGnCMsSpuRFUyCZgg=,Co/nVSLofrWsM5qpcKLXfekegArokgN29XjEXttuXK4=,wIHg6aGH2GMPX6l1pCTzeS1bXE4jxRqmd9ubES4HgW8=,ST8+q2Jgb91pWEwLwmSoJzXEGsQKeQGbzlLbgHPtj4w=,rB07AAHPffU4zFFF8IrqfKSltyWcPyy4+q+IM5SLZiQ=,WYQEP5EEzM+P+nfkHKLkGko/S3RdBgfEQ3IcyYwrChE=,fylJzYvYlM0+kRBeLB3eFKKgCibqxFvBa8hL+WStwCE=,IoM2E9pNxABFR+H3yfapUL+ThKm7GtTzY7js9H/H99o=
...,...,...,...,...,...,...,...,...,...,...
9994,jComfqYTXYSeH3GvqcOhwPmldb+BCdVJDKKDNkdtw2w=,Kr2CC15nSwDjdpyAeVh4vOuIaHuC/Q7cL9BAK28JoG8=,SRFkJlXGZgnI2svGQLoCAcrghqMRr+u5s36xzSMAOqg=,ubHy/++3FVAS97znZOt7L+cjkZFJREIiJPRZEfRIztc=,7uf0BQkpKFCgsoTY6hGENDudghJBAtKvDQ3VTc1nO7E=,lPUeS6siL3Hb9UUwnRC9piF2fYeBf+u85lUSgk4qgg4=,3yK2OPj1uYDsoMgsxsjY1FxXkOllD8Xfh20VYGqT+nU=,NMZ5RsVp7WQ9rJP2seQnLYgSaJ7ga0FV3Ieg0DW59C0=,2QaQ5ANfKi6SrWtkIv5y7DUAmEz3dJXLn5fhhjBV8N8=,uO9xKdGEeEsG0BVm6VI/0XYd8E0DRuXXEa2gwcoKHcg=
9995,Pr5enXjzVzVjZziZxrDcgnyu6CLUftmEbnp6TctyJbU=,YvZUuCDjLu9VvkCdBWgARWQrvm+FSXgxp0zIrMjcLBc=,v4GptRPKxsWXk7yKmmdhHBTqN7QoYJTK1GPjpjppMa4=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,gtvCdEuc1Tnjv6MSRDG8mAMO+KHeyqX/rg0IQwpdbi8=,v4GptRPKxsWXk7yKmmdhHBTqN7QoYJTK1GPjpjppMa4=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,zX9+hre+RQeHdvHyFAguXw2WNsshYzygopGqPn/BDLc=,v4GptRPKxsWXk7yKmmdhHBTqN7QoYJTK1GPjpjppMa4=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=
9996,9Avs0tL1zvH7Xx41z2UqrXs11/4IWLmqRAodLt/SKjQ=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,9cgVS5E58bStBXAoRa9+MN4C7HpJh+UfM6/QcCwH0k4=,FExKgjj6CsbToTubdZ+kGsOmUx3gCvZVJCdZPcdPNF4=,9Avs0tL1zvH7Xx41z2UqrXs11/4IWLmqRAodLt/SKjQ=,c5oB4c9pSRTzkd4PmhzY4BazFbmVHbhy0lbyPp7aRbA=,YvZUuCDjLu9VvkCdBWgARWQrvm+FSXgxp0zIrMjcLBc=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,zOxT5daF0yUlUsnKTZeXgFCmbLqtE7oBITbptqroU/Q=,tnLDGLnpYhzsik5+X+WPo4KQJoQA0TfWRlmEtQ3XNJQ=
9997,9zkXU3f6YnRPjsWi3lKSCLseIGrleg00tpRI4OplABw=,gOZBAoajyr6i7GgON0N7q5+KE4JTwH3OUM0lZOWMuG8=,lnHueTbAHnZiCYqByDCzhzcB9kjjnk6GOmLFNK8gG00=,Kr2CC15nSwDjdpyAeVh4vOuIaHuC/Q7cL9BAK28JoG8=,zsF/C4x766PfoC59pZccSIWFOtQtiX/RPXB76PwIvIg=,lnHueTbAHnZiCYqByDCzhzcB9kjjnk6GOmLFNK8gG00=,Kr2CC15nSwDjdpyAeVh4vOuIaHuC/Q7cL9BAK28JoG8=,kZD8nTcJKVhhRKawBwobfk93XBOLQrH0jlf74jOnMuI=,Ymx/TSp54

In [26]:
df_object_col.reset_index(drop = True , inplace  = True)

In [27]:
df_object_col["con_text"] = df_object_col["x3"].astype(str) + df_object_col["x4"].astype(str)  + df_object_col["x34"].astype(str)  + df_object_col["x35"].astype(str)  + df_object_col["x61"].astype(str)  + df_object_col["x64"].astype(str)  + df_object_col["x65"].astype(str)  + df_object_col["x91"].astype(str)  + df_object_col["x94"].astype(str)  + df_object_col["x95"].astype(str) 
df_object_col

,x3,x4,x34,x35,x61,x64,x65,x91,x94,x95,con_text
0,dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=,GNjrXXA3SxbgD0dTRblAPO9jFJ7AIaZnu/f48g5XSUk=,mimucPmJSF6NI6KM6cPIaaVxWaQyIQzSgtwTTb9bKlc=,s7mTY62CCkWUFc36AW2TlYAy5CIcniD2Vz+lHzyYCLg=,+2TNtXRI6r9owdGCS80Ia9VVv8ZpuOpVaHEvxRGGu78=,Op+X3asn5H7EQJErI7PR0NkUs3YB+Ld/8OfWuiOC8tU=,GeerC2BbPUcQfQO86NmvOsKrfTvmW7HF+Iru9y+7DPA=,bxU52teuxC05EZyzFihSiKHczE2ZAIVCXekVLG7j3C0=,+dia7tCOijlRGbABX0YKG5L85x/hXLyJwwplN5Qab04=,f4Uu1R9nnf/h03aqiRQT0Fw3WItzNToLCyRlW1Pn8Z8=,dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=GN...
1,nan,nan,l0G2rvmLGE6mpPtAibFsoW/0SiNnAuyAc4k35TrHvoQ=,lblNNeOLanWhqgISofUngPYP0Ne1yQv3QeNHqCAoh48=,MZZbXga8gvaCBqWpzrh2iKdOkcsz/bG/z4BVjUnqWT0=,TqL9cs8ZFzALzVpZv6wYBDi+6zwhrdarQE/3FH+XAlA=,aZTF/lredyP4cukeN8bh6kpBjYmS1QFNpPOg2LVm3Lg=,YvZUuCDjLu9VvkCdBWgARWQrvm+FSXgxp0zIrMjcLBc=,dsyhxXKNNJy4WVGD/v4+UGyW3jHWkx2xTdg3STsf34A=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,nannanl0G2rvmLGE6mpPtAibFsoW/0SiNnAuyAc4k35TrH...
2,ib4VpsEsqJHzDiyL0dZLQ+xQzDPrkxE+9T3mx5fv2wI=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,9TRXThP/ifDpJRGFX1LQseibUA1NJ3XM53gy+1eZ46k=,XSJ6E8aAoZC7/KAu3eETpfMg3mCq7HVBFIVIsoMKh9E=,cr+kkNnNFV9YL0vz029hk3ohIDmGuABRVNhFe0ePZyo=,oFsUwSLCWcj8UA1cqILh5afKVcvwlFA+ohJ147Wkz5I=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,mRPnGiKVOWTk/vzZaqlLXZRtdrkcQ/sX0hqBCqOuKq0=,oo9tGpHvTredpg9JkHgYbZAuxcwtSpQxU5mA/zUbxY8=,ib4VpsEsqJHzDiyL0dZLQ+xQzDPrkxE+9T3mx5fv2wI=X6...
3,BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,XQG0f+jmjLI0UHAXXH2RYL4MEHa+yd9okO+730PCZuc=,/1yAAEg6Qib4GMD+wvGOlGmpCIPIAzioWtcCwbns9/I=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,Vl+TDNSupucNoI+Fqeo7bMCkxg1hRjgTSS6NYb9BW00=,/1yAAEg6Qib4GMD+wvGOlGmpCIPIAzioWtcCwbns9/I=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=YG...
4,RTjsrrR8DTlJyaIP9Q3Z8s0zseqlVQTrlSe97GCWfbk=,3yK2OPj1uYDsoMgsxsjY1FxXkOllD8Xfh20VYGqT+nU=,XEDyQD4da6aJkZiBf+r7LD2VdhLGnCMsSpuRFUyCZgg=,Co/nVSLofrWsM5qpcKLXfekegArokgN29XjEXttuXK4=,wIHg6aGH2GMPX6l1pCTzeS1bXE4jxRqmd9ubES4HgW8=,ST8+q2Jgb91pWEwLwmSoJzXEGsQKeQGbzlLbgHPtj4w=,rB07AAHPffU4zFFF8IrqfKSltyWcPyy4+q+IM5SLZiQ=,WYQEP5EEzM+P+nfkHKLkGko/S3RdBgfEQ3IcyYwrChE=,fylJzYvYlM0+kRBeLB3eFKKgCibqxFvBa8hL+WStwCE=,IoM2E9pNxABFR+H3yfapUL+ThKm7GtTzY7js9H/H99o=,RTjsrrR8DTlJyaIP9Q3Z8s0zseqlVQTrlSe97GCWfbk=3y...
...,...,...,...,...,...,...,...,...,...,...,...
9994,jComfqYTXYSeH3GvqcOhwPmldb+BCdVJDKKDNkdtw2w=,Kr2CC15nSwDjdpyAeVh4vOuIaHuC/Q7cL9BAK28JoG8=,SRFkJlXGZgnI2svGQLoCAcrghqMRr+u5s36xzSMAOqg=,ubHy/++3FVAS97znZOt7L+cjkZFJREIiJPRZEfRIztc=,7uf0BQkpKFCgsoTY6hGENDudghJBAtKvDQ3VTc1nO7E=,lPUeS6siL3Hb9UUwnRC9piF2fYeBf+u85lUSgk4qgg4=,3yK2OPj1uYDsoMgsxsjY1FxXkOllD8Xfh20VYGqT+nU=,NMZ5RsVp7WQ9rJP2seQnLYgSaJ7ga0FV3Ieg0DW59C0=,2QaQ5ANfKi6SrWtkIv5y7DUAmEz3dJXLn5fhhjBV8N8=,uO9xKdGEeEsG0BVm6VI/0XYd8E0DRuXXEa2gwcoKHcg=,jComfqYTXYSeH3GvqcOhwPmldb+BCdVJDKKDNkdtw2w=Kr...
9995,Pr5enXjzVzVjZziZxrDcgnyu6CLUftmEbnp6TctyJbU=,YvZUuCDjLu9VvkCdBWgARWQrvm+FSXgxp0zIrMjcLBc=,v4GptRPKxsWXk7yKmmdhHBTqN7QoYJTK1GPjpjppMa4=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,gtvCdEuc1Tnjv6MSRDG8mAMO+KHeyqX/rg0IQwpdbi8=,v4GptRPKxsWXk7yKmmdhHBTqN7QoYJTK1GPjpjppMa4=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,zX9+hre+RQeHdvHyFAguXw2WNsshYzygopGqPn/BDLc=,v4GptRPKxsWXk7yKmmdhHBTqN7QoYJTK1GPjpjppMa4=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,Pr5enXjzVzVjZziZxrDcgnyu6CLUftmEbnp6TctyJbU=Yv...
9996,9Avs0tL1zvH7Xx41z2UqrXs11/4IWLmqRAodLt/SKjQ=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,9cgVS5E58bStBXAoRa9+MN4C7HpJh+UfM6/QcCwH0k4=,FExKgjj6CsbToTubdZ+kGsOmUx3gCvZVJCdZPcdPNF4=,9Avs0tL1zvH7Xx41z2UqrXs11/4IWLmqRAodLt/SKjQ=,c5oB4c9pSRTzkd4PmhzY4BazFbmVHbhy0lbyPp7aRbA=,YvZUuCDjLu9VvkCdBWgARWQrvm+FSXgxp0zIrMjcLBc=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,zOxT5daF0yUlUsnKTZeXgFCmbLqtE7oBITbptqroU/Q=,tnLDGLnpYhzsik5+X+WPo4KQJoQA0TfWRlmEtQ3XNJQ=,9Avs0tL1zvH

## Data Cleaning and Preprocessing

In [28]:
# Importing Necessary Libraries
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ankit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ankit\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ankit\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [29]:
#Creating Object of WordNet Lemmatizer
wordnet = WordNetLemmatizer()
corpus = []   #Created a List named corpus.


for i in range(0,len(df_object_col)):
    print(i)
    review = re.sub('[^a-zA-Z]' , ' ' , df_object_col['con_text'][i]) # Using Regex Function to replace digits and Special chracters with space
    review = review.lower() #converting all the reviews into lower case
    review = review.split() # splitting the reviews (by default space)
    
    # using list comprehension to compare the words which are not in stopword english library and appending them into list corpus.
    review = [wordnet.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071


3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714


5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355
5356
5357
5358
5359
5360
5361
5362
5363
5364
5365
5366


6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987
6988
6989
6990
6991
6992
6993
6994
6995
6996
6997
6998
6999
7000
7001
7002
7003
7004
7005


8447
8448
8449
8450
8451
8452
8453
8454
8455
8456
8457
8458
8459
8460
8461
8462
8463
8464
8465
8466
8467
8468
8469
8470
8471
8472
8473
8474
8475
8476
8477
8478
8479
8480
8481
8482
8483
8484
8485
8486
8487
8488
8489
8490
8491
8492
8493
8494
8495
8496
8497
8498
8499
8500
8501
8502
8503
8504
8505
8506
8507
8508
8509
8510
8511
8512
8513
8514
8515
8516
8517
8518
8519
8520
8521
8522
8523
8524
8525
8526
8527
8528
8529
8530
8531
8532
8533
8534
8535
8536
8537
8538
8539
8540
8541
8542
8543
8544
8545
8546
8547
8548
8549
8550
8551
8552
8553
8554
8555
8556
8557
8558
8559
8560
8561
8562
8563
8564
8565
8566
8567
8568
8569
8570
8571
8572
8573
8574
8575
8576
8577
8578
8579
8580
8581
8582
8583
8584
8585
8586
8587
8588
8589
8590
8591
8592
8593
8594
8595
8596
8597
8598
8599
8600
8601
8602
8603
8604
8605
8606
8607
8608
8609
8610
8611
8612
8613
8614
8615
8616
8617
8618
8619
8620
8621
8622
8623
8624
8625
8626
8627
8628
8629
8630
8631
8632
8633
8634
8635
8636
8637
8638
8639
8640
8641
8642
8643
8644
8645
8646


In [30]:
corpus

['dqoim ybygnvsezbriqxs bvofnrqrtioxrielxd g gnjrxxa sxbgd dtrblapo jfj aiaznu f g xsuk mimucpmjsf ni km cpiaavxwaqyiqzsgtwttb bklc mty cckwufc aw tlyay cicnid vz lhzyyclg tntxri r owdgcs ia vvv zpuopvahevxrggu op x asn h eqjeri pr nkus yb ld ofwuioc tu geerc bbpucqfqo nmvoskrftvmw hf iru dpa bxu teuxc ezyzfihsikhcze zaivcxekvlg j c dia tcoijlrgbabx ykg l x hxlyjwwpln qab f uu r nnf h aqirqt fw witzntolcyrlw pn z',
 'nannanl g rvmlge mpptaibfsow sinnauyac k trhvoq lblnneolanwhqgisofungpyp ne yqv qenhqcaoh mzzbxga gvacbqwpzrh ikdokcsz bg z bvjunqwt tql c zfzalzvpzv wybdi zwhrdarqe fh xala aztf lredyp cuken bh kpbjyms qfnppog lvm lg yvzuucdjlu vvkcdbwgarwqrvm fsxgxp zirmjclbc dsyhxxknnjy wvgd v ugyw jhwkx xtdg stsf x ddai dzowvu dg gcgronr vtuz mc sdhtnups',
 'ib vpsesqjhzdiyl dzlq xqzdprkxe mx fv wi x ddai dzowvu dg gcgronr vtuz mc sdhtnups trxthp ifdpjrgfx lqseibua nj xm gy ez k xsj e aaozc kau eetpfmg mcq hvbfivisomkh e cr kknnnfv yl vz hk ohidmguabrvnhfe epzyo ofsuwslcwcj ua cqilh af

In [31]:
train_df[boolean_cols]=train_df[boolean_cols].astype('category')
test_df[boolean_cols]=test_df[boolean_cols].astype('category')

In [32]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Columns: 146 entries, id to x145
dtypes: category(50), float64(55), int64(31), object(10)
memory usage: 7.8+ MB


In [33]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Columns: 146 entries, id to x145
dtypes: category(50), float64(55), int64(31), object(10)
memory usage: 1.6+ MB


In [34]:
num_imputer=SimpleImputer(strategy='mean')

In [35]:
obj_imputer=SimpleImputer(strategy='most_frequent')

In [36]:
num_imputer.fit(train_df[numeric_columns])
train_df[numeric_columns]=num_imputer.transform(train_df[numeric_columns])

In [37]:
num_imputer.fit(test_df[numeric_columns])

SimpleImputer()

In [38]:
test_df[numeric_columns]=num_imputer.transform(test_df[numeric_columns])

In [39]:
train_df[numeric_columns].isna().sum().sort_values(ascending=False)

id      0
x98     0
x111    0
x110    0
x109    0
       ..
x50     0
x49     0
x48     0
x47     0
x145    0
Length: 86, dtype: int64

In [40]:
test_df[numeric_columns].isna().sum().sort_values(ascending=False)

id      0
x98     0
x111    0
x110    0
x109    0
       ..
x50     0
x49     0
x48     0
x47     0
x145    0
Length: 86, dtype: int64

In [41]:
train_df.replace('nan', None, inplace=True)
test_df.replace('nan', None, inplace=True)

In [42]:
train_df.head()

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145
0,1.0,NO,NO,dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=,GNjrXXA3SxbgD0dTRblAPO9jFJ7AIaZnu/f48g5XSUk=,0.576561,0.073139,0.481394,0.115697,0.472474,...,0.0,0.810,3306.0,4676.0,YES,NO,YES,2.0,0.375535,0.464610
1,2.0,NaN,NaN,None,None,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.510,4678.0,3306.0,YES,NO,YES,4.0,0.741682,0.593630
2,3.0,NO,NO,ib4VpsEsqJHzDiyL0dZLQ+xQzDPrkxE+9T3mx5fv2wI=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,1.341803,0.051422,0.935572,0.041440,0.501710,...,0.0,0.850,4678.0,3306.0,NO,NO,NO,1.0,0.776467,0.493159
3,4.0,YES,NO,BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,0.653912,0.041471,0.940787,0.090851,0.556564,...,0.0,0.945,3306.0,4678.0,NO,NO,YES,3.0,0.168234,0.546582
4,5.0,NO,NO,RTjsrrR8DTlJyaIP9Q3Z8s0zseqlVQTrlSe97GCWfbk=,3yK2OPj1uYDsoMgsxsjY1FxXkOllD8Xfh20VYGqT+nU=,1.415919,0.000000,1.000000,0.000000,0.375297,...,0.0,1.000,1263.0,892.0,NO,NO,NO,1.0,0.246637,0.361045


In [43]:
for col in boolean_cols:
    train_df[col].replace({'YES':1, 'NO':0},inplace=True)
    test_df[col].replace({'YES':1, 'NO':0},inplace=True)

In [44]:
train_df[boolean_cols]

,x1,x2,x10,x11,x12,x13,x14,x24,x25,x26,...,x116,x117,x126,x127,x128,x129,x130,x140,x141,x142
0,0,0,1,0,0,0,0,1,0,1,...,0,1,1,0,0,0,0,1,0,1
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,1,0,0,0,0,1,0,1
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1,0,1,0,0,0,0,1,0,1,...,0,1,1,0,0,0,0,0,0,1
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1
9995,0,0,1,0,0,0,0,1,0,1,...,0,1,1,0,0,0,0,1,0,1
9996,1,1,1,0,0,0,0,1,0,1,...,0,0,1,0,0,0,0,1,0,1
9997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [45]:
obj_imputer.fit(train_df[boolean_cols])
train_df[boolean_cols]=obj_imputer.transform(train_df[boolean_cols])

In [46]:
obj_imputer.fit(test_df[boolean_cols])
test_df[boolean_cols]=obj_imputer.transform(test_df[boolean_cols])

In [47]:
test_df[boolean_cols]

,x1,x2,x10,x11,x12,x13,x14,x24,x25,x26,...,x116,x117,x126,x127,x128,x129,x130,x140,x141,x142
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,1,0,1,...,0,1,1,0,0,0,0,0,0,0
2,0,0,0,0,1,1,1,0,0,0,...,0,0,0,0,1,1,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0,0,1,0,0,0,0,1,0,1,...,0,1,1,0,0,0,1,1,0,1
1996,0,0,1,0,0,0,0,1,0,1,...,0,0,1,0,0,0,0,1,0,1
1997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1998,0,0,0,0,1,0,0,0,0,0,...,0,1,1,0,1,0,1,0,0,0


In [48]:
train_df[boolean_cols]

,x1,x2,x10,x11,x12,x13,x14,x24,x25,x26,...,x116,x117,x126,x127,x128,x129,x130,x140,x141,x142
0,0,0,1,0,0,0,0,1,0,1,...,0,1,1,0,0,0,0,1,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1,0,1,0,0,0,0,1,0,1,...,0,1,1,0,0,0,0,0,0,1
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1
9995,0,0,1,0,0,0,0,1,0,1,...,0,1,1,0,0,0,0,1,0,1
9996,1,1,1,0,0,0,0,1,0,1,...,0,0,1,0,0,0,0,1,0,1
9997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [49]:
train_df[boolean_cols].isna().sum().sort_values(ascending=False)

x1      0
x104    0
x74     0
x75     0
x85     0
x86     0
x87     0
x92     0
x93     0
x101    0
x102    0
x103    0
x105    0
x2      0
x115    0
x116    0
x117    0
x126    0
x127    0
x128    0
x129    0
x130    0
x140    0
x141    0
x73     0
x72     0
x71     0
x63     0
x10     0
x11     0
x12     0
x13     0
x14     0
x24     0
x25     0
x26     0
x30     0
x31     0
x32     0
x33     0
x41     0
x42     0
x43     0
x44     0
x45     0
x55     0
x56     0
x57     0
x62     0
x142    0
dtype: int64

In [50]:
train_df[boolean_cols]=train_df[boolean_cols].astype('int')
test_df[boolean_cols]=test_df[boolean_cols].astype('int')

In [51]:
train_df[numeric_columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 86 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      9999 non-null   float64
 1   x5      9999 non-null   float64
 2   x6      9999 non-null   float64
 3   x7      9999 non-null   float64
 4   x8      9999 non-null   float64
 5   x9      9999 non-null   float64
 6   x15     9999 non-null   float64
 7   x16     9999 non-null   float64
 8   x17     9999 non-null   float64
 9   x18     9999 non-null   float64
 10  x19     9999 non-null   float64
 11  x20     9999 non-null   float64
 12  x21     9999 non-null   float64
 13  x22     9999 non-null   float64
 14  x23     9999 non-null   float64
 15  x27     9999 non-null   float64
 16  x28     9999 non-null   float64
 17  x29     9999 non-null   float64
 18  x36     9999 non-null   float64
 19  x37     9999 non-null   float64
 20  x38     9999 non-null   float64
 21  x39     9999 non-null   float64
 22  

In [52]:
exclude_column = 'id'
numeric_columns = list(filter(lambda col: col != exclude_column, numeric_columns))

In [53]:
X_train = train_df[numeric_columns+boolean_cols]
X_test  = test_df[numeric_columns+boolean_cols]
Y_train = train_labels_df.loc[:9998,'y1':]

In [54]:
X_train1 = X_train.copy()
X_test1 = X_test.copy()
Y_train1 = Y_train.copy()

In [55]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)

(9999, 135)
(9999, 33)
(2000, 135)


In [56]:
X_train

,x5,x6,x7,x8,x9,x15,x16,x17,x18,x19,...,x116,x117,x126,x127,x128,x129,x130,x140,x141,x142
0,0.576561,0.073139,0.481394,0.115697,0.472474,42.0,0.396065,3.0,6.0,0.991018,...,0,1,1,0,0,0,0,1,0,1
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0,0,1,0,0,0,0,1,0,1
2,1.341803,0.051422,0.935572,0.041440,0.501710,2.0,0.838475,3.0,5.0,0.966122,...,0,0,0,0,1,0,0,0,0,0
3,0.653912,0.041471,0.940787,0.090851,0.556564,37.0,0.127405,8.0,15.0,0.959171,...,0,1,1,0,0,0,0,0,0,1
4,1.415919,0.000000,1.000000,0.000000,0.375297,1.0,0.523543,4.0,11.0,0.963004,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,1.207136,0.082855,0.918960,0.313880,0.495189,10.0,0.512852,3.0,7.0,0.967342,...,0,0,1,0,0,0,0,1,0,1
9995,1.414798,0.000000,1.000000,0.000000,0.357369,6.0,0.864350,4.0,17.0,0.974215,...,0,1,1,0,0,0,0,1,0,1
9996,1.413677,0.000000,1.000000,0.000000,0.668517,3.0,0.865471,6.0,7.0,0.939462,...,0,0,1,0,0,0,0,1,0,1
9997,1.294118,0.000000,1.000000,0.000000,0.570707,7.0,0.400871,8.0,11.0,0.949891,...,0,0,0,0,1,0,0,0,0,0


# Bag Of Words (BoW)

In [57]:
df_object_col

,x3,x4,x34,x35,x61,x64,x65,x91,x94,x95,con_text
0,dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=,GNjrXXA3SxbgD0dTRblAPO9jFJ7AIaZnu/f48g5XSUk=,mimucPmJSF6NI6KM6cPIaaVxWaQyIQzSgtwTTb9bKlc=,s7mTY62CCkWUFc36AW2TlYAy5CIcniD2Vz+lHzyYCLg=,+2TNtXRI6r9owdGCS80Ia9VVv8ZpuOpVaHEvxRGGu78=,Op+X3asn5H7EQJErI7PR0NkUs3YB+Ld/8OfWuiOC8tU=,GeerC2BbPUcQfQO86NmvOsKrfTvmW7HF+Iru9y+7DPA=,bxU52teuxC05EZyzFihSiKHczE2ZAIVCXekVLG7j3C0=,+dia7tCOijlRGbABX0YKG5L85x/hXLyJwwplN5Qab04=,f4Uu1R9nnf/h03aqiRQT0Fw3WItzNToLCyRlW1Pn8Z8=,dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=GN...
1,nan,nan,l0G2rvmLGE6mpPtAibFsoW/0SiNnAuyAc4k35TrHvoQ=,lblNNeOLanWhqgISofUngPYP0Ne1yQv3QeNHqCAoh48=,MZZbXga8gvaCBqWpzrh2iKdOkcsz/bG/z4BVjUnqWT0=,TqL9cs8ZFzALzVpZv6wYBDi+6zwhrdarQE/3FH+XAlA=,aZTF/lredyP4cukeN8bh6kpBjYmS1QFNpPOg2LVm3Lg=,YvZUuCDjLu9VvkCdBWgARWQrvm+FSXgxp0zIrMjcLBc=,dsyhxXKNNJy4WVGD/v4+UGyW3jHWkx2xTdg3STsf34A=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,nannanl0G2rvmLGE6mpPtAibFsoW/0SiNnAuyAc4k35TrH...
2,ib4VpsEsqJHzDiyL0dZLQ+xQzDPrkxE+9T3mx5fv2wI=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,9TRXThP/ifDpJRGFX1LQseibUA1NJ3XM53gy+1eZ46k=,XSJ6E8aAoZC7/KAu3eETpfMg3mCq7HVBFIVIsoMKh9E=,cr+kkNnNFV9YL0vz029hk3ohIDmGuABRVNhFe0ePZyo=,oFsUwSLCWcj8UA1cqILh5afKVcvwlFA+ohJ147Wkz5I=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,mRPnGiKVOWTk/vzZaqlLXZRtdrkcQ/sX0hqBCqOuKq0=,oo9tGpHvTredpg9JkHgYbZAuxcwtSpQxU5mA/zUbxY8=,ib4VpsEsqJHzDiyL0dZLQ+xQzDPrkxE+9T3mx5fv2wI=X6...
3,BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,XQG0f+jmjLI0UHAXXH2RYL4MEHa+yd9okO+730PCZuc=,/1yAAEg6Qib4GMD+wvGOlGmpCIPIAzioWtcCwbns9/I=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,Vl+TDNSupucNoI+Fqeo7bMCkxg1hRjgTSS6NYb9BW00=,/1yAAEg6Qib4GMD+wvGOlGmpCIPIAzioWtcCwbns9/I=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=YG...
4,RTjsrrR8DTlJyaIP9Q3Z8s0zseqlVQTrlSe97GCWfbk=,3yK2OPj1uYDsoMgsxsjY1FxXkOllD8Xfh20VYGqT+nU=,XEDyQD4da6aJkZiBf+r7LD2VdhLGnCMsSpuRFUyCZgg=,Co/nVSLofrWsM5qpcKLXfekegArokgN29XjEXttuXK4=,wIHg6aGH2GMPX6l1pCTzeS1bXE4jxRqmd9ubES4HgW8=,ST8+q2Jgb91pWEwLwmSoJzXEGsQKeQGbzlLbgHPtj4w=,rB07AAHPffU4zFFF8IrqfKSltyWcPyy4+q+IM5SLZiQ=,WYQEP5EEzM+P+nfkHKLkGko/S3RdBgfEQ3IcyYwrChE=,fylJzYvYlM0+kRBeLB3eFKKgCibqxFvBa8hL+WStwCE=,IoM2E9pNxABFR+H3yfapUL+ThKm7GtTzY7js9H/H99o=,RTjsrrR8DTlJyaIP9Q3Z8s0zseqlVQTrlSe97GCWfbk=3y...
...,...,...,...,...,...,...,...,...,...,...,...
9994,jComfqYTXYSeH3GvqcOhwPmldb+BCdVJDKKDNkdtw2w=,Kr2CC15nSwDjdpyAeVh4vOuIaHuC/Q7cL9BAK28JoG8=,SRFkJlXGZgnI2svGQLoCAcrghqMRr+u5s36xzSMAOqg=,ubHy/++3FVAS97znZOt7L+cjkZFJREIiJPRZEfRIztc=,7uf0BQkpKFCgsoTY6hGENDudghJBAtKvDQ3VTc1nO7E=,lPUeS6siL3Hb9UUwnRC9piF2fYeBf+u85lUSgk4qgg4=,3yK2OPj1uYDsoMgsxsjY1FxXkOllD8Xfh20VYGqT+nU=,NMZ5RsVp7WQ9rJP2seQnLYgSaJ7ga0FV3Ieg0DW59C0=,2QaQ5ANfKi6SrWtkIv5y7DUAmEz3dJXLn5fhhjBV8N8=,uO9xKdGEeEsG0BVm6VI/0XYd8E0DRuXXEa2gwcoKHcg=,jComfqYTXYSeH3GvqcOhwPmldb+BCdVJDKKDNkdtw2w=Kr...
9995,Pr5enXjzVzVjZziZxrDcgnyu6CLUftmEbnp6TctyJbU=,YvZUuCDjLu9VvkCdBWgARWQrvm+FSXgxp0zIrMjcLBc=,v4GptRPKxsWXk7yKmmdhHBTqN7QoYJTK1GPjpjppMa4=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,gtvCdEuc1Tnjv6MSRDG8mAMO+KHeyqX/rg0IQwpdbi8=,v4GptRPKxsWXk7yKmmdhHBTqN7QoYJTK1GPjpjppMa4=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,zX9+hre+RQeHdvHyFAguXw2WNsshYzygopGqPn/BDLc=,v4GptRPKxsWXk7yKmmdhHBTqN7QoYJTK1GPjpjppMa4=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,Pr5enXjzVzVjZziZxrDcgnyu6CLUftmEbnp6TctyJbU=Yv...
9996,9Avs0tL1zvH7Xx41z2UqrXs11/4IWLmqRAodLt/SKjQ=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,9cgVS5E58bStBXAoRa9+MN4C7HpJh+UfM6/QcCwH0k4=,FExKgjj6CsbToTubdZ+kGsOmUx3gCvZVJCdZPcdPNF4=,9Avs0tL1zvH7Xx41z2UqrXs11/4IWLmqRAodLt/SKjQ=,c5oB4c9pSRTzkd4PmhzY4BazFbmVHbhy0lbyPp7aRbA=,YvZUuCDjLu9VvkCdBWgARWQrvm+FSXgxp0zIrMjcLBc=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,zOxT5daF0yUlUsnKTZeXgFCmbLqtE7oBITbptqroU/Q=,tnLDGLnpYhzsik5+X+WPo4KQJoQA0TfWRlmEtQ3XNJQ=,9Avs0tL1zvH

In [58]:
#importing library
from sklearn.feature_extraction.text import CountVectorizer

#creating object count_vector
count_vector = CountVectorizer(ngram_range=(1, 1), lowercase = True , stop_words =  'english')
count_vector.fit_transform(df_object_col)
word_count = count_vector.get_feature_names()
word_count

['con_text',
 'x3',
 'x34',
 'x35',
 'x4',
 'x61',
 'x64',
 'x65',
 'x91',
 'x94',
 'x95']

In [59]:
#coverting text column into array
vect =  count_vector.fit_transform(df_object_col).toarray()
vect

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [60]:
frequency_matrix = pd.DataFrame((vect),columns = word_count)
frequency_matrix

,con_text,x3,x34,x35,x4,x61,x64,x65,x91,x94,x95
0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0
5,0,0,0,0,0,0,1,0,0,0,0
6,0,0,0,0,0,0,0,1,0,0,0
7,0,0,0,0,0,0,0,0,1,0,0
8,0,0,0,0,0,0,0,0,0,1,0
9,0,0,0,0,0,0,0,0,0,0,1


### Applying Bag of Words processing to our dataset

In [61]:
# Instantiate the CountVectorizer method
count_vector_data = CountVectorizer(ngram_range=(1, 1), lowercase = True , stop_words =  'english')

# Fit the training data and then return the matrix
X_train1 = count_vector_data.fit_transform(X_train1) 
# to get the the list of feature words that are lowercase with no puttutaion or stop words
#Since you fit transform X data training
# they convert data to integer data

# Transform testing data and return the matrix. Note we are not fitting the testing data into the CountVectorizer()
X_test1 = count_vector_data.transform(X_test1)
#Since you transform X data testing
# they convert data to integer data

In [62]:
X_train_feature_list = count_vector_data.get_feature_names()
X_train_feature_list

['x1',
 'x10',
 'x100',
 'x101',
 'x102',
 'x103',
 'x104',
 'x105',
 'x106',
 'x107',
 'x108',
 'x109',
 'x11',
 'x110',
 'x111',
 'x112',
 'x113',
 'x114',
 'x115',
 'x116',
 'x117',
 'x118',
 'x119',
 'x12',
 'x120',
 'x121',
 'x122',
 'x123',
 'x124',
 'x125',
 'x126',
 'x127',
 'x128',
 'x129',
 'x13',
 'x130',
 'x131',
 'x132',
 'x133',
 'x134',
 'x135',
 'x136',
 'x137',
 'x138',
 'x139',
 'x14',
 'x140',
 'x141',
 'x142',
 'x143',
 'x144',
 'x145',
 'x15',
 'x16',
 'x17',
 'x18',
 'x19',
 'x2',
 'x20',
 'x21',
 'x22',
 'x23',
 'x24',
 'x25',
 'x26',
 'x27',
 'x28',
 'x29',
 'x30',
 'x31',
 'x32',
 'x33',
 'x36',
 'x37',
 'x38',
 'x39',
 'x40',
 'x41',
 'x42',
 'x43',
 'x44',
 'x45',
 'x46',
 'x47',
 'x48',
 'x49',
 'x5',
 'x50',
 'x51',
 'x52',
 'x53',
 'x54',
 'x55',
 'x56',
 'x57',
 'x58',
 'x59',
 'x6',
 'x60',
 'x62',
 'x63',
 'x66',
 'x67',
 'x68',
 'x69',
 'x7',
 'x70',
 'x71',
 'x72',
 'x73',
 'x74',
 'x75',
 'x76',
 'x77',
 'x78',
 'x79',
 'x8',
 'x80',
 'x81',
 'x82',


In [64]:
vect_data =  X_train1.toarray()
vect_data

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [65]:
frequency_matrix_X_train = pd.DataFrame((vect_data),columns = X_train_feature_list)
frequency_matrix_X_train

,x1,x10,x100,x101,x102,x103,x104,x105,x106,x107,...,x88,x89,x9,x90,x92,x93,x96,x97,x98,x99
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
131,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
132,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
133,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# TF-IDF Vectorization

In [66]:
# importing library 
from sklearn.feature_extraction.text import TfidfVectorizer

#creating object vectorizer
vectorizer = TfidfVectorizer(max_features = 10000) #limiting the number of features to 10000
vectorized_corpus = vectorizer.fit_transform(df_object_col['con_text']).toarray() #coverting text column into array

df_temp = pd.DataFrame(vectorized_corpus,columns=vectorizer.get_feature_names_out())

print(df_temp) #printing the rows and columns

      00cweqs5zc22wnxklsmc6aqquav65  01fua6sfh369yiv9fv  02cldbveshge  \
0                               0.0                 0.0           0.0   
1                               0.0                 0.0           0.0   
2                               0.0                 0.0           0.0   
3                               0.0                 0.0           0.0   
4                               0.0                 0.0           0.0   
...                             ...                 ...           ...   
9994                            0.0                 0.0           0.0   
9995                            0.0                 0.0           0.0   
9996                            0.0                 0.0           0.0   
9997                            0.0                 0.0           0.0   
9998                            0.0                 0.0           0.0   

      02gpzxgcffnut3mztlxorpdkszrz5jlrr2l0wz0oav4  02o0k6prb6zoqxbk   03  \
0                                             0

In [67]:
print(vectorized_corpus)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [68]:
# printing count of each vectorizer
vectorizer.vocabulary_

{'gnjrxxa3sxbgd0dtrblapo9jfj7aiaznu': 3615,
 'f48g5xsuk': 3157,
 's7mty62cckwufc36aw2tlyay5cicnid2vz': 7533,
 'lhzyyclg': 5114,
 'op': 6416,
 'mzzbxga8gvacbqwpzrh2ikdokcsz': 5706,
 'bg': 1901,
 'z4bvjunqwt0': 9718,
 'aztf': 1784,
 'lredyp4cuken8bh6kpbjyms1qfnppog2lvm3lg': 5268,
 'yvzuucdjlu9vvkcdbwgarwqrvm': 9663,
 'fsxgxp0zirmjclbc': 3370,
 'dsyhxxknnjy4wvgd': 2680,
 'v4': 8489,
 'ugyw3jhwkx2xtdg3stsf34a': 8285,
 'x6ddai': 9103,
 'dzowvu0dg6gcgronr2vtuz': 2752,
 'mc4sdhtnups38': 5455,
 'ib4vpsesqjhzdiyl0dzlq': 4109,
 'xqzdprkxe': 9309,
 '9t3mx5fv2wi': 1426,
 'xsj6e8aaozc7': 9324,
 'kau3eetpfmg3mcq7hvbfivisomkh9e': 4716,
 'cr': 2309,
 'kknnnfv9yl0vz029hk3ohidmguabrvnhfe0epzyo': 4813,
 'wv5vahfyqkeuyfb5kvngfobuwjkugkyc8wh9qfpvzaa': 9016,
 'mrpngikvowtk': 5600,
 'vzzaqllxzrtdrkcq': 8762,
 'sx0hqbcqoukq0': 7753,
 'oo9tgphvtredpg9jkhgybzauxcwtspqxu5ma': 6398,
 'zubxy8': 9939,
 'ygcdisifn4flao': 9506,
 'askdzfhgiq23oqzfsbuvb6px1pig': 1717,
 'vl': 8637,
 'tdnsupucnoi': 7880,
 'fqeo7bmckxg1hr

In [69]:
# checking number of rows and columns
vectorized_corpus.shape

(9999, 10000)

# Word Embedding

In [70]:
import gensim
print("gensim version:", gensim.__version__)
from gensim.models import Word2Vec 
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')  # Download the Punkt tokenizer models

gensim version: 4.1.2


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ankit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [71]:
tokenized_texts = [word_tokenize(text.lower()) for text in df_object_col['con_text']]

# Train Word2Vec model
model = Word2Vec(sentences=tokenized_texts, vector_size=100, window=5, min_count=1, workers=4)

# Save the trained Word2Vec model
model1=model.save("word2vec_model.bin")

# Hash Field Value

In [72]:
def hash_columns(df, columns_to_hash):
    hashed_df = pd.DataFrame()

    for column in alphanumeric_cols:
        hashed_df['hashed_'+column] = df[column].apply(hash)

    return hashed_df

hashed_train_df = hash_columns(train_df,alphanumeric_cols)

hashed_test_df = hash_columns(test_df,alphanumeric_cols)

In [73]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaled_hashed_train_df = pd.DataFrame(scaler.fit_transform(hashed_train_df), columns=hashed_train_df.columns)
scaled_hashed_test_df = pd.DataFrame(scaler.transform(hashed_test_df), columns=hashed_test_df.columns)

In [74]:
scaled_hashed_train_df

,hashed_x3,hashed_x4,hashed_x34,hashed_x35,hashed_x61,hashed_x64,hashed_x65,hashed_x91,hashed_x94,hashed_x95
0,0.734872,0.641646,0.555741,0.003241,0.700335,0.898879,0.296998,0.211390,0.947629,0.761838
1,0.000000,0.000000,0.249562,0.846308,0.490325,0.917937,0.190547,0.813043,0.492349,0.240230
2,0.690138,0.240556,0.357035,0.968019,0.694723,0.199207,0.276313,0.239871,0.026819,0.182407
3,0.078844,0.834602,0.078852,0.834602,0.775354,0.136994,0.833470,0.798506,0.136954,0.833470
4,0.113687,0.890502,0.434710,0.644605,0.014297,0.301512,0.261969,0.555744,0.239548,0.899017
...,...,...,...,...,...,...,...,...,...,...
9994,0.497937,0.660045,0.314361,0.400131,0.079192,0.734022,0.889294,0.917913,0.997945,0.230042
9995,0.828467,0.814152,0.730453,0.276688,0.428486,0.730621,0.276313,0.509804,0.730407,0.276313
9996,0.501163,0.276688,0.941069,0.066853,0.501107,0.878881,0.813048,0.275976,0.040541,0.153109
9997,0.776419,0.947383,0.573805,0.660045,0.529632,0.573937,0.659150,0.054798,0.743934,0.261969


In [75]:
X_train_hash = pd.concat([X_train,scaled_hashed_train_df], axis=1)
X_test_hash = pd.concat([X_test,scaled_hashed_test_df], axis=1)

In [76]:
print(X_train_hash.shape)
print(Y_train.shape)
print(X_test_hash.shape)

(9999, 145)
(9999, 33)
(2000, 145)


In [77]:
model = OneVsRestClassifier(RandomForestClassifier())
model.fit(X_train_hash,Y_train)
Y_pred_hash = model.predict(X_train_hash)
ham_loss = hamming_loss(Y_train, Y_pred_hash)
print("Hamming Loss for Training Data",ham_loss)

Hamming Loss for Training Data 3.0306060909121216e-06


In [78]:
print("Accuracy Score for Training Data",accuracy_score(Y_train,Y_pred_hash))

Accuracy Score for Training Data 0.9998999899989999


In [79]:
classification_rep = classification_report(Y_train,Y_pred_hash)

print('Classification Report:\n', classification_rep)

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        60
           1       1.00      1.00      1.00         6
           2       1.00      1.00      1.00       220
           3       1.00      1.00      1.00       156
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00       761
           6       1.00      1.00      1.00       350
           7       1.00      1.00      1.00         6
           8       1.00      1.00      1.00       774
           9       1.00      1.00      1.00       156
          10       1.00      1.00      1.00         6
          11       1.00      1.00      1.00       742
          12       1.00      1.00      1.00       132
          13       0.00      0.00      0.00         0
          14       1.00      1.00      1.00        27
          15       1.00      1.00      1.00       112
          16       1.00      1.00      1.00         1
   

In [80]:
test_df.id

0       1698001.0
1       1698002.0
2       1698003.0
3       1698004.0
4       1698005.0
          ...    
1995    1699996.0
1996    1699997.0
1997    1699998.0
1998    1699999.0
1999    1700000.0
Name: id, Length: 2000, dtype: float64

In [81]:
target=[]
for num in test_df.id:
    for i in range(1,34):
        target.append(str(num)+'_y'+str(i))

In [82]:
len(target)

66000

In [83]:
submission_df=pd.DataFrame(target,columns=['id'])

In [84]:
submission_df

,id
0,1698001.0_y1
1,1698001.0_y2
2,1698001.0_y3
3,1698001.0_y4
4,1698001.0_y5
...,...
65995,1700000.0_y29
65996,1700000.0_y30
65997,1700000.0_y31
65998,1700000.0_y32


In [85]:
# prediction for Test data
predictions_hash = model.predict(X_test_hash)

In [86]:
prediction_hash_list=[]
for i in range(len(predictions_hash)):
    prediction_hash_list.extend(predictions_hash.tolist()[i])

In [87]:
pred_hash_df=pd.DataFrame(prediction_hash_list,columns=['pred'])

In [88]:
test_prediction_hash_df=pd.concat([submission_df,pred_hash_df],axis=1)

In [89]:
test_prediction_hash_df.to_csv('test_prediction_final1.csv',index=None)